In [1]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, Activation, MaxPooling2D,MaxPooling2D,AveragePooling2D, Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.layers import Dropout, Flatten, Concatenate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
import keras_resnet.models
import keras

Using TensorFlow backend.


In [2]:
df = pd.read_csv('legend.csv',header = None)


In [3]:
X = np.stack([cv2.imread('images\\'+str(img)) for img in df.iloc[5:905,1]]).astype(np.float)[:,:,:,np.newaxis]
x_t = np.stack([cv2.imread('images\\'+str(img1)) for img1 in df.iloc[905:1205,1]]).astype(np.float)[:,:,:,np.newaxis]

In [4]:
y = np.vstack(df.iloc[5:905,-1].values)
y_t = np.vstack(df.iloc[905:1205,-1].values)

In [5]:
y_train = y.reshape(900,)
y_test = y_t.reshape(300,)


In [6]:
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y1 = np_utils.to_categorical(encoded_Y)
encoder.fit(y_test)
encoded_Y_t = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y2 = np_utils.to_categorical(encoded_Y_t)

In [7]:
X_train = X / 255
X_test = x_t / 255


In [8]:
X_train = X_train.reshape(900,350,350,3)
X_test = X_test.reshape(300,350,350,3)

In [ ]:
first = Sequential()
first.add(BatchNormalization(batch_size = 5,input_shape=(350,350,3)))
first.add(Conv2D(8, kernel_size=(2,2), strides=(1, 1), padding='same', data_format='channels_last', activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))

first.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

first.add(Conv2D(8, kernel_size=(2,2), strides=(1, 1), padding='same', data_format='channels_last', activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))

first.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
first.add(Dropout(0.2))
first.add(Conv2D(16, kernel_size=(2,2), strides=(1, 1), padding='same', data_format='channels_last', activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))

first.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
first.add(Dropout(0.2))
first.add(Conv2D(32, kernel_size=(2,2), strides=(1, 1), padding='same', data_format='channels_last', activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))

first.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
first.add(Dropout(0.4))
first.add(Flatten())

#sgd = optimizers.SGD(lr=0.000001, decay=0.0005, momentum=0.95, nesterov=False)

#first.compile(loss='mse', optimizer = sgd, metrics=['accuracy'])




first.add(Dense(50, activation="relu"))


first.add(Dense(6,activation='softmax'))
first.summary()

first.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


history = first.fit(X_train, y1, batch_size=5, epochs=20, verbose=1, validation_split=0.1, shuffle=True)
score = first.evaluate(X_test, y2, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (5, 350, 350, 3)          12        
_________________________________________________________________
conv2d_5 (Conv2D)            (5, 350, 350, 8)          104       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (5, 175, 175, 8)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (5, 175, 175, 8)          264       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (5, 87, 87, 8)            0         
_________________________________________________________________
dropout_4 (Dropout)          (5, 87, 87, 8)            0         
_________________________________________________________________
conv2d_7 (Conv2D)            (5, 87, 87, 16)           528       
__________